In [328]:
import os
import pandas as pd
import pdfplumber
import re

In [ ]:
# exception caused by rare cases of race data that overflow page boundaries. data not yet complete

In [329]:
def extract_votes_from_row(data_row,header):
    votes_ls = re.search(r'[0-9]+\s[0-9]+\s[0-9]+\s[0-9]+',data_row).group(0).split(' ')
    votes = {}
    for i in range(0,len(header)):
        votes[header[i]] = [votes_ls[i]]
    df = pd.DataFrame(votes)
    df['candidate'] = re.search(r'[^0-9]*',data_row).group(0).strip()
    return df

def extract_votes(data_rows,header):
    if len(data_rows) > 0:
        return pd.concat([extract_votes_from_row(data_row,header) for data_row in data_rows])
    return None

def extract_data_rows(table_rows):
    data_rows = []
    for row in table_rows:
        if not re.search(r'[0-9]+\s[0-9]+\s[0-9]+\s[0-9]+',row) is None:
            data_rows.append(row)

    return data_rows

def extract_box_data(page,bbox):
    data_section = page.crop(bbox,strict=False)
    data_text = data_section.extract_text()
    table_rows = data_text.split('\n')
    race_title = table_rows[0]

    table_header = ['Votes','ED','MI','PR']
    
    data_rows = extract_data_rows(table_rows)

    df = extract_votes(data_rows,table_header)

    if not re.search(r'\(([A-Za-z]{,3})\)',race_title) is None:
        df['party'] = re.search(r'\(([A-Za-z]{,3})\)',race_title).group(1)
    if not re.search(r'^[^\(]+',race_title) is None:
        df['office'] = re.search(r'^[^\(]+',race_title).group(0).strip()

    return df

def extract_precinct_header(page):
    if not re.search(r'(Precinct .*)\n',page.extract_text()) is None:
        return re.search(r'(Precinct .*)\n',page.extract_text()).group(1)
    return None

def extract_precinct_name(precinct_header):
    return re.search(r'[^0-9]*',precinct_header).group(0).strip()

def extract_data_bounding_boxes(page,vote_headers):
    all_boxes = []
    i = 0
    while i < len(vote_headers):
        if i < len(vote_headers) - 1:
            pair = vote_headers[i:i+2]
            bbox = (0,pair[0]['top']-0,page.width,pair[1]['top']-0)
        else:
            bbox = (0,vote_headers[i]['top']-0,page.width,page.height)
        all_boxes.append(bbox)
        i += 1
    return all_boxes

def extract_page_data(page):
    vote_headers = page.search('Vote for')
    if (len(vote_headers)) > 0:
        all_data = []
        bounding_boxes = extract_data_bounding_boxes(page,vote_headers)
        box_datums = list(filter(lambda x: x is not None,[extract_box_data(page,bbox) for bbox in bounding_boxes]))
        if len(box_datums) > 0:
            df = pd.concat(box_datums)
            return df
    return None

def extract_statistics(page):
    if len(page.search('STATISTICS')) > 0:
        bbox = (0,page.search('STATISTICS')[0]['bottom'],page.width,page.search('STATISTICS')[0]['bottom'] + 150)
        stats_text = page.crop(bbox).extract_text()

        stats = pd.DataFrame({
            'Registered Voters': [re.search(r'Registered Voters - Total ([0-9]*)',stats_text).group(1)],
            'Ballots Cast': [re.search(r'Ballots Cast - Total ([0-9]*)',stats_text).group(1)]
        })

        stats = stats.melt().rename(columns={
            'variable':'office',
            'value':'votes'
        })

        stats['precinct'] = extract_precinct_name(page)

        return stats
    else:
        return None

In [330]:
file = '../data_2024/primary/Bucks PA County Final EMS Report for 2024 General Primary. Detail and Grand total.pdf'
pdf = pdfplumber.open(file)
county_name = 'Bucks'

In [331]:
all_data = []
for page in pdf.pages:
    precinct_header = extract_precinct_header(page)
    if not precinct_header is None:
        split_line = page.search('Precinct ')[0]['top']
        above_header = page.crop((0,0,page.width,split_line))
        below_header = page.crop((0,split_line,page.width,page.height))

        temp = extract_page_data(above_header)
        if not temp is None:
            temp['precinct'] = precinct
            all_data.append(temp)

        precinct = extract_precinct_name(precinct_header)

        temp = extract_page_data(below_header)
        if not temp is None:
            temp['precinct'] = precinct
            all_data.append(temp)
    else:
        temp = extract_page_data(page)
        if not temp is None:
            temp['precinct'] = precinct
            all_data.append(temp)
df = pd.concat(all_data)

In [332]:
#df = pd.concat([extract_page_data(page) for page in pdf.pages])

In [333]:
df['county'] = county_name

In [334]:
df = df.rename(columns={
    'Votes':'votes',
    'ED':'election_day',
    'PR':'provisional',
    'MI':'absentee'
})

In [335]:
df = df[df['candidate'].apply(lambda x: x not in ['Total'])]

In [336]:
# get the office name up until the first dash
df['office'] = df['office'].apply(lambda x: re.search(r'(.*) -',x).group(1).strip() if not re.search(r'.* -',x) is None else x)
# pull out the district number
df['district'] = df['office'].apply(lambda x: re.search(r'([0-9]+)[a-z]+',x).group(1) if not re.search(r'[0-9]+[a-z]+',x) is None else None)

In [337]:
# strip the district numbers off the end of the race names
df['office'] = df['office'].apply(lambda x: re.search(r'(.*)[0-9]+[a-z]+',x).group(1) if not re.search(r'(.*)[0-9]+[a-z]+',x) is None else x)
df['office'] = df['office'].apply(lambda x: re.sub('[0-9]+$','',x).strip())

In [338]:
df['party'] = df['party'].str.upper()
df = df.fillna('')
df = df.drop_duplicates()
df = df.reset_index()
df = df[['county','precinct','office','district','party','candidate','votes','election_day','provisional','absentee']]

In [339]:
# data cleaning
df['candidate'] = df['candidate'].str.title()
df['office'] = df['office'].str.replace('Delagate','Delegate')
df['office'] = df['office'].replace('Presidential Electors','President')
df['office'] = df['office'].replace('United States Senator','U.S. Senate')
df['office'] = df['office'].replace('Representative in Congress','U.S. House')
df['office'] = df['office'].replace('Senator in the General Assembly','State Senate')
df['office'] = df['office'].replace("Representative in the General Assembly ",'General Assembly')

In [16]:
df.to_csv(f'../data_cleaned/20240423__pa__primary__{county_name.lower()}__precinct.csv',index=False)